In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 13.2 MB/s eta 0:00:00


In [3]:
try:
  from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op as frontend_op  # pylint:disable=g-import-not-at-top
except ImportError:
  frontend_op = None

In [4]:
from typing import Tuple, Optional

import tensorflow as tf
from tensorflow import keras
import tensorflow_io as tfio

import numpy as np
import pandas as pd

import os
import sys
import tarfile
import hashlib
import re
import glob

import random
import math

import IPython.display as ipd
from tensorflow.python.util import compat

In [5]:
LIB_PATH = '/content/drive/MyDrive/GSC/GSC_helper'
sys.path.append(LIB_PATH)
from utils import _download, unzipzip, zipzip
from GSC import download_GSC
from GSC12 import SpeechCommands12

## GSC

In [7]:
ZIP_MAP = download_GSC('https://drive.google.com/file/d/1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-5Bglq0ihfzZ4tM_zZ6dxAsnoYv1YzAm/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-6St6zZqFvDy0JHhHTNV9UORH0mTjoDp/view?usp=drive_link',
                       '/content/GSC_8',
                       end = '.zip')
CSV_MAP = download_GSC('https://drive.google.com/file/d/1-GQ55fRsP1zNat93Yoc2tM7tUqUPiuch/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-FIGYCKCiKdTqZjcdbqVdKTUBt6-R1w9/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-Mpiyo-AE5at9ahnO8qD9ljl75lr7DJA/view?usp=drive_link',
                       '/content/GSC_8',
                       end = '.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf
From (redirected): https://drive.google.com/uc?id=1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf&confirm=t&uuid=c70bf638-aaec-4d06-a3e3-310ca5f95c66
To: /content/GSC_8/train.zip
100%|██████████| 58.6M/58.6M [00:00<00:00, 67.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-5Bglq0ihfzZ4tM_zZ6dxAsnoYv1YzAm
To: /content/GSC_8/val.zip
100%|██████████| 5.66M/5.66M [00:00<00:00, 211MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-6St6zZqFvDy0JHhHTNV9UORH0mTjoDp
To: /content/GSC_8/test.zip
100%|██████████| 6.34M/6.34M [00:00<00:00, 128MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-GQ55fRsP1zNat93Yoc2tM7tUqUPiuch
To: /content/GSC_8/train.csv
100%|██████████| 600k/600k [00:00<00:00, 83.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-FIGYCKCiKdTqZjcdbqVdKTUBt6-R1w9
To: /content/GSC_8/val.csv
100%|██████████| 59.0k/59.0k [00:00<00:00, 37.5MB/s]
Downloading...
From: h

In [8]:
import pandas as pd
import os

class GSC_8(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,
                 root: str,
                 zip_map: dict,
                 csv_map: dict,
                 unzip: bool = True,
                 subset: str = 'train',
                 batch_size: int = 32,
                 shuffle: bool = True):
        'Initialization'
        super().__init__()
        local_path = os.path.join(root, subset)
        self.root = root
        if not os.path.exists(local_path):
            os.mkdir(local_path)
            unzipzip(zip_map[subset], local_path)
        if unzip:
            unzipzip(zip_map[subset], local_path)
        self.csv = pd.read_csv(csv_map[subset])
        self.subset = subset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the numbber of batches per epoch'
        return int(np.floor(len(self.csv)/self.batch_size))

    def __getitem__(self, index):
        'Generate on batch of data'
        # Generate indexes of the batcch
        indexes = self.indexes[index*self.batch_size: (index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Undates indexes after each epoch'
        self.indexes = np.arange(len(self.csv))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X: (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i in indexes:
            # Store sample
            row = self.csv.iloc[i]
            X.append(np.load(os.path.join(self.root, row['link']))['arr_0'])

            # Store class
            y.append(row['label'])
        X = np.stack(X, axis = 0)
        y = np.stack(y, axis = 0)
        return tf.convert_to_tensor(X), tf.convert_to_tensor(y)

In [9]:
train_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'train', batch_size = 128, shuffle = True)
val_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'val', batch_size = 128, shuffle = False)
test_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'test', batch_size = 10, shuffle = False)

Extracted /content/GSC_8/train.zip
Extracted /content/GSC_8/val.zip
Extracted /content/GSC_8/test.zip


In [10]:
X, y = next(iter(train_dataloader))
X.shape

TensorShape([128, 49, 40, 1])

In [11]:
model = keras.models.Sequential([
    keras.layers.Input(shape = (49, 40, 1)),
    keras.layers.Conv2D(3, kernel_size = 5, strides = 2, padding = 'same', activation = 'relu', use_bias = False),
    keras.layers.Conv2D(3, kernel_size = 3, strides = 1, padding = 'same', groups = 3, use_bias = False),
    keras.layers.Conv2D(16, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(16, kernel_size = 3, strides = 2, padding = 'same', groups = 16, use_bias = False),
    keras.layers.Conv2D(32, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(8),
    keras.layers.Softmax()])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 20, 3)         75        
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 20, 3)         27        
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 20, 16)        48        
                                                                 
 batch_normalization (Batch  (None, 25, 20, 16)        64        
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 25, 20, 16)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 10, 16)        144       
                                                        

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0015),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [13]:
EPOCHS = 20
history = model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=EPOCHS,
    #callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

Epoch 1/20
170/170 [==============================] - 83s 472ms/step - loss: 1.9875 - accuracy: 0.1854 - val_loss: 1.8475 - val_accuracy: 0.2207
Epoch 2/20
170/170 [==============================] - 83s 490ms/step - loss: 1.8052 - accuracy: 0.2642 - val_loss: 1.6623 - val_accuracy: 0.3277
Epoch 3/20
170/170 [==============================] - 80s 469ms/step - loss: 1.5852 - accuracy: 0.3854 - val_loss: 1.3643 - val_accuracy: 0.4672
Epoch 4/20
170/170 [==============================] - 78s 459ms/step - loss: 1.3865 - accuracy: 0.4793 - val_loss: 1.1291 - val_accuracy: 0.5855
Epoch 5/20
170/170 [==============================] - 83s 489ms/step - loss: 1.2849 - accuracy: 0.5251 - val_loss: 1.0195 - val_accuracy: 0.6316
Epoch 6/20
170/170 [==============================] - 80s 470ms/step - loss: 1.1906 - accuracy: 0.5671 - val_loss: 0.8144 - val_accuracy: 0.7297
Epoch 7/20
170/170 [==============================] - 82s 481ms/step - loss: 1.0872 - accuracy: 0.6055 - val_loss: 0.7812 - val_ac

In [14]:
model.evaluate(test_dataloader)

292/292 [==============================] - 4s 15ms/step - loss: 0.6960 - accuracy: 0.7781


[0.6959914565086365, 0.7780821919441223]

## BCResNet2TFLite

In [15]:
from tqdm import tqdm

test_specs = []
test_labels = []

for i in tqdm(range(len(test_dataloader.csv))):
    row = test_dataloader.csv.iloc[i]
    spec = np.load(os.path.join('/content/GSC_8', row['link']))['arr_0']
    test_specs.append(spec)
    test_labels.append(row['label'])

test_specs = tf.convert_to_tensor(np.stack(test_specs, axis = 0))
test_labels = tf.convert_to_tensor(np.stack(test_labels))

100%|██████████| 2925/2925 [00:02<00:00, 1312.79it/s]


In [16]:
test_specs.shape

TensorShape([2925, 49, 40, 1])

In [17]:
test_labels.shape

TensorShape([2925])

In [18]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(test_specs).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint 8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [20]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/gsc_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the dynamic range quantized model:
tflite_model_quant_file = tflite_models_dir/"gsc_tflite_backup_model3_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

274464

In [21]:
tflite_model_file = tflite_models_dir/"gsc_model_backup3.tflite"
tflite_model_file.write_bytes(tflite_model)

1074688

In [22]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
    global test_specs

    # Initialize the interpreter
    interpreter = tf.lite.Interpreter(model_path = str(tflite_file))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    predictions = np.zeros((len(test_image_indices), ), dtype = int)
    for i, test_image_index in enumerate(test_image_indices):
        test_image = test_specs[test_image_index]

        # Check if the input type is quantized, the rescale input data to to uint8
        if input_details['dtype'] == np.int8:
            input_scale, input_zero_point = input_details['quantization']
            test_image = test_image/input_scale + input_zero_point

        test_image = np.expand_dims(test_image, axis = 0).astype(input_details['dtype'])
        interpreter.set_tensor(input_details['index'], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])[0]

        predictions[i] = output.argmax()

    return predictions

# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_specs
  global test_labels

  test_image_indices = range(test_specs.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_specs)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_specs)))

In [23]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

Quantized model accuracy is 77.4359% (Number of test samples=2925)


In [24]:
evaluate_model(tflite_model_file, model_type="Float32")

Float32 model accuracy is 77.7778% (Number of test samples=2925)


In [25]:
import shutil
shutil.copy2(str(tflite_model_quant_file), '/content')

'/content/gsc_tflite_backup_model3_quant.tflite'

In [26]:
# Save the file as a C source file
!xxd -i /content/gsc_tflite_backup_model3_quant.tflite > /content/bkup_model_conv.cc
# Print the source file
!cat /content/bkup_model_conv.cc

Streaming output truncated to the last 5000 lines.
  0xf7, 0xf9, 0x04, 0x04, 0xfc, 0x04, 0xff, 0xfc, 0x01, 0x02, 0xfd, 0xf9,
  0x01, 0xfa, 0xfe, 0xfa, 0x01, 0xfe, 0xfd, 0x03, 0xf8, 0x00, 0x03, 0x01,
  0x03, 0x03, 0xff, 0x02, 0xf9, 0xfa, 0x01, 0x05, 0xfa, 0x04, 0xf7, 0x05,
  0xff, 0xf7, 0xfd, 0x05, 0x02, 0x04, 0x00, 0xf8, 0x03, 0x00, 0xff, 0x01,
  0xfe, 0xff, 0xff, 0xfd, 0xff, 0xf7, 0x00, 0x02, 0x03, 0xff, 0xff, 0x05,
  0x01, 0xfa, 0xfe, 0x02, 0x01, 0x06, 0x02, 0x05, 0x04, 0xfb, 0xf8, 0x00,
  0x05, 0x04, 0xfc, 0x00, 0xfc, 0x03, 0xfa, 0x03, 0x04, 0x01, 0x04, 0xfd,
  0xfa, 0xfa, 0xfc, 0xf8, 0xf8, 0xf9, 0xfd, 0xfb, 0xff, 0x05, 0xf7, 0xfa,
  0xf9, 0x04, 0xfd, 0x03, 0xf8, 0xfb, 0xfa, 0xff, 0xfc, 0xfc, 0xfd, 0xf9,
  0xfc, 0x05, 0xf8, 0x00, 0x02, 0x01, 0xfe, 0xfc, 0xfd, 0xfc, 0xf8, 0xff,
  0xf8, 0xf8, 0x06, 0x02, 0x04, 0xf8, 0xf8, 0xf7, 0xfb, 0x05, 0xfb, 0xfd,
  0xfb, 0x00, 0x05, 0x04, 0xff, 0xfb, 0xfe, 0x01, 0x04, 0xf9, 0xf8, 0x04,
  0xfe, 0x06, 0xf9, 0x01, 0xfa, 0xfa, 0xf9, 0xfd, 0x01, 0x06,